In [380]:
#import function from part1
%run Part_1.ipynb

Word: Estuvimos, Label: O
Word: hace, Label: O
Word: poco, Label: O
Word: mi, Label: O
Word: pareja, Label: O
Word: y, Label: O
Word: yo, Label: O
Word: comiendo, Label: O
Word: y, Label: O
Word: resultó, Label: O
Word: todo, Label: O
Word: muy, Label: O
Word: bien, Label: O
Word: ,, Label: O
Word: tanto, Label: O
Word: la, Label: O
Word: comida, Label: B-positive
Word: ,, Label: O
Word: el, Label: O
Word: vino, Label: B-positive
Word: ,, Label: O
Word: el, Label: O
Word: trato, Label: B-positive
Word: ,, Label: O
Word: la, Label: O
Word: decoración, Label: B-positive
Word: …, Label: O
Word: nos, Label: O
Word: gustó, Label: O
Word: todo, Label: O
Word: mucho, Label: O
Word: ., Label: O
Word: Por, Label: O
Word: poner, Label: O
Word: algún, Label: O
Word: pero, Label: O
Word: ,, Label: O
Word: quizá, Label: O
Word: el, Label: O
Word: jamón, Label: B-negative
Word: no, Label: O
Word: era, Label: O
Word: todo, Label: O
Word: lo, Label: O
Word: ", Label: O
Word: ibérico, Label: O
Word: ",

In [381]:
#Estimates the transition parameters from the training set using MLE
def estimate_transition_data(training_data):
    transition_counts={}
    tags_counts={}
    for sentence in training_data:

        start_tag = sentence[0][1]
        transition_counts[('start', start_tag)] = transition_counts.get(('start', start_tag), 0) + 1
        tags_counts['start'] = tags_counts.get('start', 0) + 1
        
        stop_tag=sentence[-1][1]
        transition_counts[(stop_tag, 'stop')] = transition_counts.get((stop_tag, 'stop'), 0) + 1
        tags_counts[stop_tag] = tags_counts.get(stop_tag, 0) + 1

        for i in range(0,len(sentence)-1):
            first_tag,second_tag = sentence[i][1],sentence[i+1][1]
            transition_counts[(first_tag, second_tag)] = transition_counts.get((first_tag, second_tag), 0) + 1
            tags_counts[first_tag] = tags_counts.get(first_tag, 0) + 1
    
    transition_parameters={}
    for (first,second), count in transition_counts.items():
        tags_count = tags_counts[first]
        transition_parameters[(first,second)]=count/tags_count

    return transition_parameters

transition_parameters = estimate_transition_data(training_data)

for (first, second), param in transition_parameters.items():
    
    print(f"First: {first}, Second: {second}, Transition Parameter: {param:.4f}")

First: start, Second: O, Transition Parameter: 0.9289
First: O, Second: stop, Transition Parameter: 0.0634
First: O, Second: O, Transition Parameter: 0.8857
First: O, Second: B-positive, Transition Parameter: 0.0365
First: B-positive, Second: O, Transition Parameter: 0.8716
First: O, Second: B-negative, Transition Parameter: 0.0122
First: B-negative, Second: O, Transition Parameter: 0.8110
First: start, Second: B-positive, Transition Parameter: 0.0522
First: O, Second: B-neutral, Transition Parameter: 0.0021
First: B-neutral, Second: I-neutral, Transition Parameter: 0.2083
First: I-neutral, Second: I-neutral, Transition Parameter: 0.6512
First: I-neutral, Second: O, Transition Parameter: 0.3488
First: B-positive, Second: I-positive, Transition Parameter: 0.1164
First: I-positive, Second: I-positive, Transition Parameter: 0.5701
First: I-positive, Second: O, Transition Parameter: 0.4268
First: B-neutral, Second: O, Transition Parameter: 0.7917
First: B-negative, Second: I-negative, Tran

In [382]:
#Implement the Viterbi algorithm
def viterbi_algo(sentences, tags, trans_prob, emit_prob):
    n = len(sentences)
    count_tags = len(tags)

    #Used to store the maximum probability of each state at each timestep
    # Each dictionary element corresponds to a timestep
    # key:state,value:maximum probability of arriving at the state from the starting state
    pi = [{}]

    #Used to store the optimal path for each state at the current timestep
    #key:state,value:optimal path
    path = {}

    #Initialization
    # if "start" in tags:
    #     pi[0]["start"] = 1.0
    #     path["start"] = ["start"]
    
    for tag in tags:
        # if tag != "start":
        pi[0][tag] = 0.0  # Initialize with zero probability
        path[tag] = []

        if trans_prob.get(('start',tag),0.0)!=0:
            pi[0][tag] = trans_prob.get('start',tag)
            
    #Recurision
    

    for i in range(n):
        pi.append({})
        new_path = {}

        for u in tags:
            
            max_prob = -1
            max_prev_tag = None

            for v in tags:
                if i == 0:
                    prev_prob = 1.0
                else:
                    prev_prob = pi[i][v]
                trans = trans_prob.get((v,u),0.0)
                emit = emit_prob.get((sentences[i],u),0.0)
                prob = prev_prob * trans * emit

                if prob > max_prob:
                    max_prob = prob
                    max_prev_tag = v

            pi[i + 1][u] = max_prob
            new_path[u] = path[max_prev_tag] + [u] #update optimal path for current state in current timestep
        
        path = new_path #update the path after each timestep
        # print(path)

    #Final Step
    max_prob = -1
    max_prev_tag = None
    for v in tags:
        final_prob = pi[n][v] * trans_prob.get((v,'stop'),0.0)
        if final_prob > max_prob:
            max_prob = final_prob
            max_prev_tag = v

    final_max_tag = max_prev_tag

    return path[final_max_tag]


In [383]:
# For all the datasets RU, ES, learn these parameters with train

es_train_path = 'Data/ES/train'
ru_train_path = 'Data/RU/train'

es_training_data = read_training_data(es_train_path)
ru_training_data = read_training_data(ru_train_path)

es_emission_parameters = estimate_emission_parameters(es_training_data,k)
ru_emission_parameters = estimate_emission_parameters(ru_training_data,k)

es_transition_parameters = estimate_transition_data(es_training_data)
ru_transition_parameters = estimate_transition_data(ru_training_data)



In [384]:
#Run the Viterbi algorithm on ES
es_dev_in = read_dev_in('Data/ES/dev.in')

es_tags=()
for (first, second), param in es_transition_parameters.items():
    if first not in es_tags and first!='start':
        es_tags=es_tags+(first,)
    if second not in es_tags and second!='start':
        es_tags=es_tags+(second,)
print(es_tags)

es_predicted_tags=[]

for sentences in es_dev_in:
    path = viterbi_algo(sentences, es_tags, es_transition_parameters, es_emission_parameters)
    # print(path)
    for i in range(len(sentences)):
        es_predicted_tags.append((sentences[i],path[i]))

#write in dev_out
write_predicted_labels_to_file(es_predicted_tags, 'Data/ES/dev.p2.out')




('O', 'stop', 'B-positive', 'B-negative', 'B-neutral', 'I-neutral', 'I-positive', 'I-negative')


In [385]:
#Run the Viterbi algorithm on RU
ru_dev_in = read_dev_in('Data/RU/dev.in')

ru_tags=()
for (first, second), param in ru_transition_parameters.items():
    if first not in ru_tags and first!='start':
        ru_tags=ru_tags+(first,)
    if second not in es_tags and second!='start':
        ru_tags=ru_tags+(second,)
print(ru_tags)

ru_predicted_tags=[]

for sentences in ru_dev_in:
    path = viterbi_algo(sentences, ru_tags, ru_transition_parameters, ru_emission_parameters)
    # print(path)
    for i in range(len(sentences)):
        ru_predicted_tags.append((sentences[i],path[i]))

#write in dev_out
write_predicted_labels_to_file(ru_predicted_tags, 'Data/RU/dev.p2.out')

('O', 'B-positive', 'I-positive', 'B-negative', 'I-negative', 'B-neutral', 'I-neutral', '.. O', '... O')


In [386]:
# Read gold-standard outputs
es_gold_standard_outputs = read_gold_standard_outputs('Data/ES/dev.out')
ru_gold_standard_outputs = read_gold_standard_outputs('Data/RU/dev.out')

# Calculate scores for ES dataset
es_precision, es_recall, es_f_score = calculate_scores(es_predicted_tags, es_gold_standard_outputs)
print("For ES Dataset:")
print(f"Precision: {es_precision:.2f}")
print(f"Recall: {es_recall:.2f}")
print(f"F-score: {es_f_score:.2f}")

# Calculate scores for RU dataset
ru_precision, ru_recall, ru_f_score = calculate_scores(ru_predicted_tags, ru_gold_standard_outputs)
print("\nFor RU Dataset:")
print(f"Precision: {ru_precision:.2f}")
print(f"Recall: {ru_recall:.2f}")
print(f"F-score: {ru_f_score:.2f}")

For ES Dataset:
Precision: 0.09
Recall: 0.02
F-score: 0.04

For RU Dataset:
Precision: 0.07
Recall: 0.02
F-score: 0.03
